In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%cd ../../

C:\Projects\python\recommender


In [3]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp

In [4]:
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHA_PATH = Path("./inputs/topcoder/challenge_sm.csv")
regs_min = 4
cha_min = 4

In [5]:
regs_df = pd.read_csv(REGS_PATH)
regs_df.head()

,challengeId,registant,date
0,30044052,phead,2014-07-09
1,30044052,lovefreya,2014-07-09
2,30044052,TMALBONPH,2014-07-09
3,30044052,daga_sumit,2014-07-09
4,30044052,abcivashritt,2014-07-09


## Add Challenge DataFrame

In [6]:
from ast import literal_eval

In [7]:
attr_df = pd.read_csv("./inputs/topcoder/challenge.csv",
                      converters={
                          'technologies': literal_eval,
                          'platforms': literal_eval
                      })
attr_df.head()

,challengeId,date,prizes,technologies,platforms
0,30044052,2014-07-09,1800.0,[HTML],[HTML]
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]"
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS]
3,30044055,2014-07-09,400.0,[iOS],[iOS]
4,30044056,2014-07-09,400.0,[iOS],[iOS]


In [8]:
chag_df = regs_df[['challengeId']]
chag_df.shape

(610025, 1)

In [9]:
chag_df = chag_df.drop_duplicates(subset=['challengeId'])
chag_df.shape

(39916, 1)

In [10]:
chag_df = pd.merge(left=chag_df, right=attr_df, how='inner', on='challengeId')
chag_df.head()

,challengeId,date,prizes,technologies,platforms
0,30044052,2014-07-09,1800.0,[HTML],[HTML]
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]"
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS]
3,30044055,2014-07-09,400.0,[iOS],[iOS]
4,30044056,2014-07-09,400.0,[iOS],[iOS]


In [11]:
chag_df['per'] = chag_df.date.str[:7]
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
0,30044052,2014-07-09,1800.0,[HTML],[HTML],2014-07
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]",2014-07
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS],2014-07
3,30044055,2014-07-09,400.0,[iOS],[iOS],2014-07
4,30044056,2014-07-09,400.0,[iOS],[iOS],2014-07


In [13]:
pd.Period(value='2005-1')

Period('2005-01', 'M')

In [14]:
pd.Timestamp('2005-01-01 00:00:00')

Timestamp('2005-01-01 00:00:00')

In [15]:
chag_df['date'][0]

Timestamp('2014-07-09 00:00:00')

In [12]:
chag_df.loc[-1] = (-1, '2005-01-01', 0, [], [], '2005-01')

In [13]:
chag_df = chag_df.sort_index()
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
-1,-1,2005-01-01,0.0,[],[],2005-01
0,30044052,2014-07-09,1800.0,[HTML],[HTML],2014-07
1,30044053,2014-07-09,1250.0,"[Salesforce, Apex, Visualforce]","[Force.com, Salesforce.com]",2014-07
2,30044054,2014-07-09,400.0,"[Java, MySQL, REST]",[AWS],2014-07
3,30044055,2014-07-09,400.0,[iOS],[iOS],2014-07


## Encode for period

In [16]:
from sklearn.preprocessing import OrdinalEncoder

In [20]:
chag_df = chag_df.sort_values(by=['per'])
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
-1,-1,2005-01-01,0.0,[],[],2005-01
31190,30000000,2006-10-20,0.0,[],[],2006-10
31189,30000002,2006-10-20,0.0,[],[],2006-10
31188,30000018,2006-10-30,0.0,[],[],2006-10
31139,30000136,2006-11-16,0.0,"[.NET, C#]",[],2006-11


In [18]:
period_encoder = OrdinalEncoder(categories='auto')
period_encoder.fit(chag_df[['per']])

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [19]:
batch_df = chag_df.sample(n=3)
batch_df.head()

,challengeId,date,prizes,technologies,platforms,per
21498,30038341,2013-01-24,350.0,[],[],2013-01
8366,30056177,2017-01-16,1000.0,[Other],[Other],2017-01
9270,30057354,2017-11-01,9000.0,[],[],2017-11


In [20]:
chag_df.head()

,challengeId,date,prizes,technologies,platforms,per
31190,30000000,2006-10-20,0.0,[],[],2006-10
31189,30000002,2006-10-20,0.0,[],[],2006-10
31188,30000018,2006-10-30,0.0,[],[],2006-10
31187,30000030,2006-11-04,0.0,"[Java, HTTP, JSF]",[],2006-11
31095,30000196,2006-11-23,0.0,[Java],[],2006-11


In [21]:
period_encoder.transform(chag_df.head()[['per']])

array([[0.],
       [0.],
       [0.],
       [1.],
       [1.]])

## Encode for technologies

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

In [23]:
tech_bin = MultiLabelBinarizer(sparse_output=True)
tech_bin.fit(chag_df['technologies'])

MultiLabelBinarizer(classes=None, sparse_output=True)

In [24]:
batch_df = chag_df.sample(n=10)
batch_df.head()

,challengeId,date,prizes,technologies,platforms,per
396,30044593,2014-07-28,150.0,"[HTML, JavaScript, Node.js]",[NodeJS],2014-07
13192,30062149,2018-01-18,5.0,[Other],[Other],2018-01
10934,30059505,2017-09-22,20.0,[],[],2017-09
35958,30006200,2009-03-19,600.0,"[Web Services, Flex]",[],2009-03
19773,30041980,2014-04-17,1500.0,[],[],2014-04


In [25]:
tech_bin.transform(batch_df['technologies'].repeat(2))

<20x216 sparse matrix of type '<class 'numpy.int32'>'
	with 20 stored elements in Compressed Sparse Row format>

## Preprocess Registers Dataframe

In [14]:
regs_df = pd.read_csv(REGS_PATH)
regs_df.head()

,challengeId,registant,date
0,30044052,phead,2014-07-09
1,30044052,lovefreya,2014-07-09
2,30044052,TMALBONPH,2014-07-09
3,30044052,daga_sumit,2014-07-09
4,30044052,abcivashritt,2014-07-09


In [15]:
regs_counts = regs_df['registant'].value_counts()
regs_counts.head()

sunbinbrother    7451
poundinc_tc      6992
woodjhon         6398
luckvin          6039
Schpotsky        5757
Name: registant, dtype: int64

In [16]:
regs_counts.shape

(60017,)

In [17]:
regs_counts = regs_counts[regs_counts >= 4]
regs_counts.shape

(9894,)

In [18]:
regs_df.shape

(610025, 3)

In [19]:
regs_df = regs_df[regs_df['registant'].isin(regs_counts.index)]
regs_df.shape

(544568, 3)

In [20]:
regs_df['period'] = regs_df['date'].str[:7]
regs_df.head()

,challengeId,registant,date,period
0,30044052,phead,2014-07-09,2014-07
1,30044052,lovefreya,2014-07-09,2014-07
2,30044052,TMALBONPH,2014-07-09,2014-07
3,30044052,daga_sumit,2014-07-09,2014-07
4,30044052,abcivashritt,2014-07-09,2014-07


In [21]:
regs_df = regs_df.sort_values(by=['registant', 'date'])
regs_df.head()

,challengeId,registant,date,period
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09
482515,30000149,-Neo-,2006-11-16,2006-11


In [22]:
regs_df['previousId'] = regs_df['challengeId']
regs_df['prevPeriod'] = regs_df['period']
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,30022003,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30045145,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045639,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045678,2014-09
482515,30000149,-Neo-,2006-11-16,2006-11,30000149,2006-11


In [23]:
regs_df['previousId'] = regs_df['previousId'].shift(
    periods=1).fillna(0).astype('int64')
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,0,2010-03
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30022003,2014-08
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045145,2014-09
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045639,2014-09
482515,30000149,-Neo-,2006-11-16,2006-11,30045678,2006-11


In [24]:
regs_df['prevPeriod'] = regs_df['prevPeriod'].shift(periods=1).fillna('2005-01')
regs_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,0,2005-01
10253,30045145,(acm)zhupeijun,2014-08-19,2014-08,30022003,2010-03
16217,30045639,(acm)zhupeijun,2014-09-05,2014-09,30045145,2014-08
16674,30045678,(acm)zhupeijun,2014-09-08,2014-09,30045639,2014-09
482515,30000149,-Neo-,2006-11-16,2006-11,30045678,2014-09


In [25]:
first_mask = regs_df.duplicated(subset=['registant'], keep='first')
first_mask.head()

446754    False
10253      True
16217      True
16674      True
482515    False
dtype: bool

In [26]:
regs_df['previousId'] = regs_df['previousId'].where(first_mask, -1)
regs_df[~first_mask].head()

,challengeId,registant,date,period,previousId,prevPeriod
446754,30022003,(acm)zhupeijun,2010-03-17,2010-03,-1,2005-01
482515,30000149,-Neo-,2006-11-16,2006-11,-1,2014-09
166877,30055022,-_-33,2016-08-19,2016-08,-1,2007-08
588599,30003421,-jacob-,2008-01-24,2008-01,-1,2016-12
588099,30003490,-kotenok-,2008-01-31,2008-01,-1,2014-07


## Batch collate function

In [36]:
batch_df = regs_df.sample(n=200)
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
344253,30035744,gwiwik,2013-09-18,2013-09,30035551,2013-09
514674,30012744,sparemax,2010-07-09,2010-07,30012746,2010-07
515460,30012542,azukun,2010-08-03,2010-08,30013718,2010-08
219230,30059401,solokop,2017-09-18,2017-09,30059397,2017-09
312526,30041080,testXuSanping,2014-03-07,2014-03,30041101,2014-03


In [39]:
batch_df = batch_df.sort_values(by=['period'])
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
482626,30000143,singlewood,2006-11-16,2006-11,-1,2010-03
608521,30000621,_NT_,2007-01-11,2007-01,30000578,2007-01
602737,30001496,knuthocean,2007-04-24,2007-04,30001467,2007-04
597470,30002173,CDDB,2007-07-27,2007-07,30002071,2007-07
596971,30002240,StargateKeeper,2007-08-06,2007-08,30002241,2007-08


In [38]:
batch_df = batch_df.sort_values(by=['previousId'])
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
482626,30000143,singlewood,2006-11-16,2006-11,-1,2010-03
582256,30004416,prettyangels,2008-05-06,2008-05,-1,2018-08
506955,30013984,dljg718,2010-08-30,2010-08,-1,2013-04
448303,30021858,bargisy,2009-12-16,2009-12,-1,2007-10
395517,30029234,weligamage,2012-08-08,2012-08,-1,2010-07


In [41]:
prev_feat_df = pd.merge(left=batch_df[['previousId']],
                        right=chag_df,
                        how='left',
                        left_on=['previousId'],
                        right_on=['challengeId'])
prev_feat_df.head()

,previousId,challengeId,date,prizes,technologies,platforms,per
0,-1,-1,2005-01-01,0.0,[],[],2005-01
1,30000578,30000578,2007-01-05,800.0,"[.NET, C#]",[],2007-01
2,30001467,30001467,2007-04-20,500.0,[Java],[],2007-04
3,30002071,30002071,2007-07-13,700.0,"[Java, XML]",[],2007-07
4,30002241,30002241,2007-08-06,400.0,[Java],[],2007-08


In [42]:
prev_feat_df.shape

(200, 7)

In [34]:
prev_feat_df = prev_feat_df.sort_values(by=['per', 'challengeId'])
prev_feat_df.head()

,previousId,challengeId,date,prizes,technologies,platforms,per
0,-1,-1,2005-01-01,0.0,[],[],2005-01
1,-1,-1,2005-01-01,0.0,[],[],2005-01
2,-1,-1,2005-01-01,0.0,[],[],2005-01
3,-1,-1,2005-01-01,0.0,[],[],2005-01
4,-1,-1,2005-01-01,0.0,[],[],2005-01


In [43]:
pos_feat_df = pd.merge(left=batch_df[['challengeId']],
                       right=chag_df,
                       how='inner',
                       on=['challengeId'])
pos_feat_df.head()

,challengeId,date,prizes,technologies,platforms,per
0,30000579,2007-01-05,800.0,"[.NET, C#]",[],2007-01
1,30000581,2007-01-03,1400.0,"[.NET, C#, WinForms Controls]",[],2007-01
2,30001211,2007-03-22,800.0,[Java],[],2007-03
3,30001952,2007-06-21,500.0,"[Java, JDBC]",[],2007-06
4,30001966,2007-06-28,500.0,"[Java, JSP, HTML, HTTP, Custom Tag, JSF, JavaS...",[],2007-06


In [44]:
per_counts = batch_df['period'].value_counts().sort_index()
per_counts.head()

2007-01    2
2007-03    1
2007-06    2
2007-08    2
2008-01    1
Name: period, dtype: int64

In [47]:
neg_list = [
    chag_df[chag_df['per'] == per].sample(n=per_counts[per] *
                                             2,
                                             replace=True)
    for per in per_counts.index
]
neg_df = pd.concat(neg_list)
neg_df.head()

,challengeId,date,prizes,technologies,platforms,per
39759,30000614,2007-01-11,0.0,"[Java, JDBC]",[],2007-01
39784,30000580,2007-01-03,1400.0,"[.NET, C#, WinForms Controls]",[],2007-01
39683,30000751,2007-01-27,500.0,"[.NET, C#]",[],2007-01
39766,30000600,2007-01-05,0.0,[.NET],[],2007-01
39417,30001204,2007-03-23,0.0,"[XSL, C#, .NET 2.0, SQL Server, ADO.NET]",[],2007-03


In [46]:
from sklearn.preprocessing import OneHotEncoder

In [47]:
chag_encoder = OneHotEncoder(categories='auto', handle_unknown='error')
chag_encoder.fit(regs_df[['challengeId']])

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [48]:
batch_df = regs_df.sample(n=5)
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
143249,30053784,sac11,2016-04-16,2016-04,30053134,2016-04
539839,30008431,googlesmkm,2009-11-17,2009-11,30008371,2009-11
195180,30056740,krisjen,2017-03-23,2017-03,30056136,2017-01
268074,30066204,linuxuser,2018-06-09,2018-06,30066106,2018-06
166460,30055011,DaraK,2016-08-15,2016-08,30054986,2016-08


In [75]:
batch_df.shape

(200, 6)

In [76]:
batch_df = batch_df.sort_values(by=['period'])
batch_df.head()

,challengeId,registant,date,period,previousId,prevPeriod
608687,30000598,DanLazar,2007-01-05,2007-01,-1,2013-09
598584,30002010,A20,2007-07-05,2007-07,30002011,2007-07
598622,30002004,wang_liang,2007-07-05,2007-07,30001978,2007-06
597049,30002223,kurtrips,2007-08-03,2007-08,30002181,2007-07
588651,30003411,amit_pundir,2008-01-22,2008-01,30003403,2008-01


In [82]:
prev_feat_df = pd.merge(left=batch_df[['previousId']],
                        right=chag_df,
                        how='inner',
                        left_on=['previousId'],
                        right_on=['challengeId'])
print("Get")
prev_feat_df = prev_feat_df.sort_values(by=['per'])
print("Get")
prev_feat_df.head()

Get


TypeError: Cannot compare type Period with type list

In [66]:
per_counts = batch_df['period'].value_counts().sort_index()
per_counts.head()

2009-11    1
2016-04    1
2016-08    1
2017-03    1
2018-06    1
Freq: M, Name: period, dtype: int64

In [67]:
neg_list = [
    chag_df[chag_df['per'] == per].sample(n=per_counts[per])
    for per in per_counts.index
]
neg_df = pd.concat(neg_list)
neg_df.head()

,challengeId,date,prizes,technologies,platforms,per
34838,30008328,2009-11-13,650.0,[.NET 3.5],[],2009-11
6708,30053593,2016-04-01,0.0,"[Apex, Visualforce, Force.com Sites]","[Force.com, Salesforce.com]",2016-04
7559,30054911,2016-08-04,1400.0,"[CSS, HTML5]",[HTML],2016-08
8762,30056704,2017-03-29,1700.0,[],[],2017-03
16092,30066177,2018-06-08,300.0,[Android],[Android],2018-06


In [49]:
chag_vector = chag_encoder.transform(batch_df[['challengeId']])
chag_vector

<5x38394 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [50]:
chag_vector.data

array([1., 1., 1., 1., 1.])

In [51]:
chag_vector = sp.hstack([chag_vector, chag_vector])
chag_vector.shape

(5, 76788)

In [52]:
chag_vector

<5x76788 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in COOrdinate format>

In [53]:
cols = chag_vector.indices
cols

AttributeError: indices not found

In [54]:
batch_df[['challengeId']].values.repeat(2, axis=0)

array([[30053784],
       [30053784],
       [30008431],
       [30008431],
       [30056740],
       [30056740],
       [30066204],
       [30066204],
       [30055011],
       [30055011]], dtype=int64)

In [34]:
batch_df['registant'].repeat(2)

109963        naeem95m
109963        naeem95m
274846         feiyusk
274846         feiyusk
100732    m-v-kachalov
100732    m-v-kachalov
423947            caru
423947            caru
355629    lanchongyizu
355629    lanchongyizu
Name: registant, dtype: object